In [48]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.chat_models.openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

In [49]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [50]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=500)

In [51]:
#Carregar o PDF

pdf_link = "lol.pdf"

loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [52]:
len(pages)

35

In [53]:
# Splitter
child_splitter = RecursiveCharacterTextSplitter(chunk_size = 200)

parent_splitter = RecursiveCharacterTextSplitter(chunk_size = 4000, chunk_overlap=200, length_function=len, add_start_index = True)

In [54]:
#Storages
store = InMemoryStore()
vectordb = Chroma(embedding_function=embeddings, persist_directory='child3.db')

In [55]:
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectordb,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)

parent_document_retriever.add_documents(pages, ids=None)

In [56]:
parent_document_retriever.vectorstore.get()

{'ids': ['87321dce-1d5b-47ca-96f9-a27d2bfa4184',
  'e4fe0ef9-8984-47e3-ae86-1773215727a4',
  '7a83cdef-cd64-4fc3-9421-47e0fe3c9fc5',
  '18366ee5-9c88-4cf2-9322-be362f6b8fa1',
  'b2cda21c-d6b6-421f-87fa-c32670122df5',
  'd5a4d91a-793e-4a56-b002-29247df5c4d5',
  '42eba4aa-173b-446f-b2a7-6afac941c2b6',
  'e52e84ad-32c7-41c1-b223-5c3190ede31a',
  'f874f451-0eec-4051-ad92-809b89a82ad6',
  'b9cece75-cd6c-4d50-8595-244bb25e803e',
  '009a6cbe-df8e-4276-ba18-125eb0331241',
  'b1446c0e-edd8-4854-9951-091c2b1afd3f',
  '1ef81116-6860-4a26-9a9a-bb7542998a6f',
  '0c0c84f4-61fb-47a9-b0f4-12613cc1fe8e',
  '0073c208-4c4d-43fa-a655-74efe4c9dfb0',
  'ba9ebc9e-d85a-4824-aa5d-f5a77b9ab29e',
  '530ee71a-d534-461c-bbc6-b1f9e6233e33',
  'f49ae1b3-b82b-40aa-8ac4-d8781f9aa74b',
  'a1843e86-be6c-4fd4-801f-64ca7b09bc26',
  '5c9b54f9-b88f-4eae-9c55-c8d847561784',
  '3655061b-64c7-41a5-ae6f-0fccf63168ea',
  '421559ea-c9e9-4af7-892d-71e265b700af',
  '4fc6270a-f351-4dc5-89e8-59d3f021a019',
  '9eb56ecc-2e97-4b72-a03f-

In [61]:
TEMPLATE = """"
    Você é um especialista em league of legends. Responda a pergunta abaixo utilizando o contexto informado.

    Query:
    {question}

    Context:
    {context}
"""

rag_promp = ChatPromptTemplate.from_template(TEMPLATE)

In [62]:
setup_retrieval = RunnableParallel({
    "question": RunnablePassthrough(),
    "context": parent_document_retriever
})

output_parser = StrOutputParser()

In [63]:
parent_chain_retrieval = setup_retrieval | rag_promp | llm | output_parser

In [65]:
parent_chain_retrieval.invoke("Como posso subir rapidamente de elo?")

'Para subir rapidamente de elo no League of Legends, é importante focar em farmar corretamente e construir itens de forma eficiente. Além disso, é fundamental ganhar mais jogos do que perder, para aumentar o seu MMR e assim ganhar mais pontos e perder menos nas derrotas. Jogar com campeões fáceis também pode ser uma estratégia eficaz, já que eles são projetados para facilitar a jogabilidade e ajudar os jogadores a atingir seus objetivos de forma mais fluida. Lembre-se de sempre ficar um passo à frente dos outros jogadores e focar em melhorar suas habilidades e conhecimento do jogo.'